<a href="https://colab.research.google.com/github/Reggiewright97/Neural-Network-Challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [62]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense, Input

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [64]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [65]:
# Create y_df with the Attrition and Department columns
columns_list = [
    "Education",
    "Age",
    "DistanceFromHome",
    "JobSatisfaction",
    "OverTime",
    "StockOptionLevel",
    "WorkLifeBalance",
    "YearsAtCompany",
    "YearsSinceLastPromotion",
    "NumCompaniesWorked",
]


In [66]:
# Create a list of at least 10 column names to use as X data
X_df = attrition_df[columns_list]




# Create X_df using your selected columns
X_df = attrition_df[columns_list]




# Show the data types for X_df
print(X_df.dtypes)


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [67]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_df,
    y_df,
    test_size=0.2,
    random_state=1
)



In [68]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
overtime_label_encoder = LabelEncoder()
overtime_label_encoder.fit(X_train['OverTime'])

X_train['OverTime'] = overtime_label_encoder.transform(X_train['OverTime'])
X_test['OverTime'] = overtime_label_encoder.transform(X_test['OverTime'])

X_train['OverTime'].value_counts()

,count
OverTime,
0,855
1,321


In [69]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [70]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
department_encoder = OneHotEncoder(sparse_output=False)


# Fit the encoder to the training data
department_encoder.fit_transform(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data

y_train_department_encoded = department_encoder.transform(y_train[['Department']])
y_test_department_encoded = department_encoder.transform(y_test[['Department']])


In [71]:
# @title Default title text
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)


# Fit the encoder to the training data
attrition_encoder.fit_transform(y_train[["Attrition"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])



In [72]:
print(y_train_department_encoded.shape)
print(y_train_attrition_encoded.shape)

(1176, 3)
(1176, 2)


## Create, Compile, and Train the Model

In [74]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,), name= "input")

# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation="relu", name= "shared_layer_1")(input_layer)
shared_layer_2 = layers.Dense(128, activation="relu", name= "shared_layer_2")(shared_layer_1)

In [75]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation="relu", name= "department_hidden_layer")(shared_layer_2)

# Create the output layer
department_output_layer = layers.Dense(y_train_department_encoded.shape[1], activation="softmax", name= "department_output")(department_hidden_layer)


In [76]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = Dense(32, activation="relu", name= "attrition_hidden")(shared_layer_2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition_encoded.shape[1], activation= "sigmoid", name= "attrition_output")(attrition_hidden_layer)


In [82]:
# Create the model
model = Model(
    inputs = input_layer,
    outputs = [department_output_layer, attrition_output_layer]
)


# Compile the model
model.compile(
    optimizer="adam",
    loss={
        "department_output": "categorical_crossentropy",
        "attrition_output": "binary_crossentropy",
    },
    metrics={
        "department_output": "accuracy",
        "attrition_output": "accuracy",
    }
)


# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_1 (Dense)    │ (None, 64)             │            704 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_2 (Dense)    │ (None, 128)            │          8,320 │ shared_layer_1[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden_layer   │ (None, 32)             │          4,128 │ shared_layer_2[0][0]   │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared_layer_2[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden_lay… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [83]:
# Train the model
model.fit(X_train_scaled,
          [y_train_department_encoded, y_train_attrition_encoded],
          validation_data=(X_test_scaled, [y_test_department_encoded, y_test_attrition_encoded]),
          epochs=100,
          verbose=2
)

Epoch 1/100
37/37 - 3s - 71ms/step - attrition_output_accuracy: 0.8095 - attrition_output_loss: 0.4730 - department_output_accuracy: 0.6173 - department_output_loss: 0.5813 - loss: 1.0556 - val_attrition_output_accuracy: 0.8027 - val_attrition_output_loss: 0.4503 - val_department_output_accuracy: 0.6327 - val_department_output_loss: 0.4890 - val_loss: 0.9486
Epoch 2/100
37/37 - 0s - 6ms/step - attrition_output_accuracy: 0.8665 - attrition_output_loss: 0.3780 - department_output_accuracy: 0.6590 - department_output_loss: 0.4863 - loss: 0.8653 - val_attrition_output_accuracy: 0.8231 - val_attrition_output_loss: 0.4370 - val_department_output_accuracy: 0.6327 - val_department_output_loss: 0.4725 - val_loss: 0.9149
Epoch 3/100
37/37 - 0s - 3ms/step - attrition_output_accuracy: 0.8682 - attrition_output_loss: 0.3528 - department_output_accuracy: 0.6590 - department_output_loss: 0.4798 - loss: 0.8331 - val_attrition_output_accuracy: 0.8197 - val_attrition_output_loss: 0.4405 - val_department

In [87]:
# Evaluate the model with the testing data
evaluation = model.evaluate(X_test_scaled, [y_test_department_encoded, y_test_attrition_encoded])

evaluation

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.7502 - attrition_output_loss: 2.0717 - department_output_accuracy: 0.5573 - department_output_loss: 1.3936 - loss: 3.4777 


[3.305162191390991,
 1.3308786153793335,
 1.9057095050811768,
 0.7687074542045593,
 0.5442177057266235]

In [86]:
# Print the accuracy for both department and attrition
print('Department Accuracy: %.3f' % evaluation[4])
print('Attrition Accuracy: %.3f' % evaluation[3])


Department Accuracy: 0.544
Attrition Accuracy: 0.769


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?
2. What activation functions did you choose for your output layers, and why?
3. Can you name a few ways that this model might be improved?



YOUR ANSWERS HERE

1. While accuracy can provide a quick overview of model performance, it may not be the most informative metric, especially in cases of class imbalance or when the costs of different types of errors vary.

2. I used 2 choices of activation functions - one for department classification and another for attrition prediction.

3. Improving a model is often an iterative process that involves experimentation and evaluation. By systematically applying these strategies, you might be able to enhance the model's performance.